In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings("ignore")

In [2]:
from springstone.data import get_data, get_missing_dates, create_train_test, create_df_for_prophet, download_model
from springstone.utils import bollinger_bands, moving_average, prophet_preprocessing, prophet_non_business_days
from springstone.utils import basic_recommendation, temp_data_predict, next_business_day
from springstone.trainer import Trainer
from springstone.predict import predict_from_model
import pandas as pd
import numpy as np
from prophet import Prophet
from prophet.plot import plot_forecast_component_plotly
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
import joblib
from termcolor import colored
from datetime import date, timedelta
import holidays

In [3]:
ticker = 'AAPL'

In [4]:
hist = get_data(ticker, end='2022-03-12')

[*********************100%***********************]  1 of 1 completed


In [5]:
hist

,Open,High,Low,Close,Volume
Date,,,,,
2012-02-21,18.102858,18.387501,18.004286,18.387501,605595200
2012-02-22,18.324286,18.410357,18.181070,18.322857,483302400
2012-02-23,18.395714,18.493929,18.196428,18.442499,568027600
2012-02-24,18.559643,18.674999,18.522858,18.657499,415072000
2012-02-27,18.618214,18.875000,18.438572,18.777143,547582000
...,...,...,...,...,...
2022-03-07,163.360001,165.020004,159.039993,159.300003,96418800
2022-03-08,158.820007,162.880005,155.800003,157.440002,131148300
2022-03-09,161.479996,163.410004,159.410004,162.949997,91454900


In [6]:
df_nbd = prophet_non_business_days(hist)

In [7]:
# Using the Trainer class
df_train, df_test = create_train_test(hist)
trainer_prophet = Trainer(model="prophet",X=df_train, y=None, non_business_days=df_nbd)
df_test_prophet = create_df_for_prophet(df_test)
trainer_prophet.run()
trainer_prophet.save_model_locally(ticker,'prophet')

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Initial log joint probability = -43.7746
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       7268.53     0.0130693       1031.53           1           1      124   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       7321.36    0.00159364       362.424           1           1      243   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     278       7330.59    0.00123625       363.072   5.172e-06       0.001      383  LS failed, Hessian reset 
     299       7333.22     0.0145204       843.769      0.3809           1      407   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     319       7335.82   0.000116299       176.341   1.536e-06       0.001      468  LS failed, Hessian reset 
     383       7337.62   0.000115938       138.322   2.581e-06       0.001      585  LS failed, Hessian rese

In [13]:
df_test

,Open,High,Low,Close,Volume
Date,,,,,
2020-03-11,69.347504,70.305000,67.964996,68.857498,255598800
2020-03-12,63.985001,67.500000,62.000000,62.057499,418474000
2020-03-13,66.222504,69.980003,63.237499,69.492500,370732000
2020-03-16,60.487499,64.769997,60.000000,60.552502,322423600
2020-03-17,61.877499,64.402496,59.599998,63.215000,324056000
...,...,...,...,...,...
2022-03-07,163.360001,165.020004,159.039993,159.300003,96418800
2022-03-08,158.820007,162.880005,155.800003,157.440002,131148300
2022-03-09,161.479996,163.410004,159.410004,162.949997,91454900


In [32]:
#y_pred = trainer_prophet.predict(df_test_prophet[:5][['ds']])
y_test_pred = predict_from_model(ticker, 'prophet', df_test_prophet[['ds']])

In [33]:
pd.concat([df_test[['Close']], pd.DataFrame({'Close_pred': y_test_pred.values}, index=df_test.index)], axis=1)

,Close,Close_pred
Date,,
2020-03-11,68.857498,65.608732
2020-03-12,62.057499,65.610120
2020-03-13,69.492500,65.595568
2020-03-16,60.552502,65.781116
2020-03-17,63.215000,65.857129
...,...,...
2022-03-07,159.300003,93.753985
2022-03-08,157.440002,93.754331
2022-03-09,162.949997,93.818996


In [34]:
next_business_day(date.today())

datetime.date(2022, 3, 14)

In [ ]:
n_days_prediction=1
future = pipeline.make_future_dataframe(periods=n_days_prediction)
future

In [ ]:
forecast = pipeline.predict(future)

In [ ]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'non business day']]

In [ ]:
m_prophet.plot(forecast);

In [ ]:
m_prophet.plot_components(forecast);

In [ ]:
#plot_forecast_component_plotly(m_prophet, forecast, 'non business day')

In [ ]:
forecast = forecast[- n_days_prediction:][['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'non business day']]

In [ ]:
forecast

In [ ]:
df_test[:n_days_prediction]

In [ ]:
mae_prophet = mean_absolute_error(df_test[:n_days_prediction]['y'], forecast['yhat'])
mape_prophet = mean_absolute_percentage_error(df_test[:n_days_prediction]['y'], forecast['yhat'])
print(f'MAE: {mae_prophet}')
pbrint(f'MAPE: {mape_prophet}')

In [ ]:
def predict_from_model(ticker, model_type, X):
    """Predicts from a saved model for a ticker and a model_type
       Input:
            ticker: ticker used by the model
            model_type: type of model used such as prohet or RNN
            X: data
       Ouptut: return a dataframe with predicted values"""
    model = download_model(ticker, model_type)
    if model_type == "prophet":
        y_pred = model['prophet_model'].prophet.predict(X)['yhat']
    else:
        y_pred = model.predict(X)
        
    return y_pred

In [ ]:
cd notebooks

In [ ]:
download_model(ricker, 'prophet')